In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [4]:
with open('pages-ubat.json', ) as fopen:
    urls = sorted(json.load(fopen))
len(urls)

772

In [3]:
!mkdir page-ubat

In [11]:
for no, url in tqdm(enumerate(urls)):
    filename = os.path.join('page-ubat', f'{no}.html')
    if os.path.exists(filename):
        continue
    
    url = f'https://hellodoktor.com{url}'
    content = requests.get(url).content.decode()
    with open(filename, 'w') as fopen:
        fopen.write(content)

772it [17:14,  1.34s/it]


In [14]:
url

'https://hellodoktor.com/ubat/zzzquil-bm/'

In [13]:
from glob import glob

files = glob('page-ubat/*.html')
len(files)

772

In [16]:
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup

TIKA_HOST = 'http://localhost:9998'


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(string, minlen=2,):
    while True:
        try:
            raw_xml = parser.from_buffer(string, TIKA_HOST, xmlContent=True)
            break
        except BaseException:
            time.sleep(1.0)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass

    return t

In [17]:
with open(files[0]) as fopen:
    t = fopen.read()

In [27]:
with open('ubat.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            t = fopen.read()
        t = '\n'.join(parse_tika(t))
        fopen_l.write(f'{json.dumps(t)}\n')

100%|███████████████████████████████████████████████████████████████████████████████████████████| 772/772 [00:14<00:00, 51.56it/s]
